In [5]:
import os

file_path = "/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/emar_detail.csv"

file_size_bytes = os.path.getsize(file_path)
file_size_mb = file_size_bytes / (1024 ** 2)

print(f"📦 حجم الملف: {file_size_mb:.2f} ميغابايت ({file_size_bytes:,} بايت)")


📦 حجم الملف: 5192.38 ميغابايت (5,444,600,366 بايت)


pandas.read_csv() مع chunksize

In [4]:
import pandas as pd
import time
import psutil
import os


file_path = "/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/emar_detail.csv"


chunk_size = 50000  


start_time = time.time()


process = psutil.Process(os.getpid())
start_memory = process.memory_info().rss / (1024 ** 2)


reader = pd.read_csv(file_path, chunksize=chunk_size)

total_rows = 0
for i, chunk in enumerate(reader):
    total_rows += len(chunk)


end_time = time.time()
elapsed_time = end_time - start_time


end_memory = process.memory_info().rss / (1024 ** 2)
used_memory = end_memory - start_memory


print(f"\n📊 تم الانتهاء من قراءة الملف بالكامل بعد {i+1} جزء.")
print(f"📦 مجموع الصفوف المقروءة: {total_rows}")
print(f"⏱️ الوقت المستغرق: {elapsed_time:.2f} ثانية")
print(f"💾 الذاكرة المستخدمة تقريبًا: {used_memory:.2f} MB")


/tmp/ipykernel_270/1708427206.py:23: DtypeWarning: Columns (11,25,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):
/tmp/ipykernel_270/1708427206.py:23: DtypeWarning: Columns (9,11,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):
/tmp/ipykernel_270/1708427206.py:23: DtypeWarning: Columns (11,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):
/tmp/ipykernel_270/1708427206.py:23: DtypeWarning: Columns (8,11,21,23,25,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):
/tmp/ipykernel_270/1708427206.py:23: DtypeWarning: Columns (11,14,28,29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):
/tmp/ipykernel_270/1708427206.py:2


📊 تم الانتهاء من قراءة الملف بالكامل بعد 1091 جزء.
📦 مجموع الصفوف المقروءة: 54514587
⏱️ الوقت المستغرق: 213.43 ثانية
💾 الذاكرة المستخدمة تقريبًا: 29.78 MB


dask

In [5]:
import dask.dataframe as dd
import warnings
import time
import psutil

warnings.filterwarnings("ignore", category=RuntimeWarning)

file_path = "/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/emar_detail.csv"

# الأعمدة ذات المشاكل تُقرأ كنصوص
dtype_fix = {
    'completion_interval': 'object',
    'continued_infusion_in_other_location': 'object',
    'dose_due': 'object',
    'dose_given': 'object',
    'infusion_complete': 'object',
    'infusion_rate_adjustment': 'object',
    'infusion_rate_unit': 'object',
    'new_iv_bag_hung': 'object',
    'non_formulary_visual_verification': 'object',
    'product_amount_given': 'object',
    'product_description_other': 'object',
    'reason_for_no_barcode': 'object',
    'restart_interval': 'object',
    'route': 'object',
    'side': 'object',
    'site': 'object',
    'infusion_rate': 'object',
    'prior_infusion_rate': 'object',
    'infusion_rate_adjustment_amount': 'object'
}

start_time = time.time()
start_memory = psutil.Process().memory_info().rss / (1024 ** 2)


df = dd.read_csv(file_path, dtype=dtype_fix, assume_missing=True)

print("✅ أول 5 صفوف من البيانات:")
print(df.head())

# 🔢 حساب المتوسطات للأعمدة الرقمية فقط
numeric_means = df.select_dtypes(include=['number']).mean().compute()
print("\n📊 متوسطات الأعمدة الرقمية:\n", numeric_means)

end_time = time.time()
end_memory = psutil.Process().memory_info().rss / (1024 ** 2)

execution_time_dask = end_time - start_time
used_memory_dask = end_memory - start_memory

print(f"\n⏱️ وقت التنفيذ الكلي: {execution_time_dask:.2f} ثانية")
print(f"💾 الذاكرة المستخدمة أثناء التنفيذ: {used_memory_dask:.2f} ميغابايت")


✅ أول 5 صفوف من البيانات:
   subject_id      emar_id  emar_seq  parent_field_ordinal  \
0  10000032.0  10000032-10      10.0                   1.1   
1  10000032.0  10000032-10      10.0                   1.2   
2  10000032.0  10000032-10      10.0                   1.3   
3  10000032.0  10000032-10      10.0                   1.4   
4  10000032.0  10000032-10      10.0                   NaN   

               administration_type  pharmacy_id barcode_type  \
0                             <NA>   48770010.0           if   
1                             <NA>   48770010.0           if   
2                             <NA>   48770010.0           if   
3                             <NA>   48770010.0           if   
4  Standard Maintenance Medication          NaN         <NA>   

  reason_for_no_barcode complete_dose_not_given dose_due  ...  \
0                  <NA>                    <NA>     <NA>  ...   
1                  <NA>                    <NA>     <NA>  ...   
2                  <N

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/usr/


📊 متوسطات الأعمدة الرقمية:
 subject_id              1.499642e+07
emar_seq                9.477730e+02
parent_field_ordinal    1.127324e+00
pharmacy_id             5.003770e+07
dtype: float64

⏱️ وقت التنفيذ الكلي: 197.25 ثانية
💾 الذاكرة المستخدمة أثناء التنفيذ: 1845.58 ميغابايت


Compression

أولا ضغط الملف

In [1]:
import gzip
import shutil

input_file = "/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/emar_detail.csv"
output_file = "/kaggle/working/emar_detail.csv.gz"  

with open(input_file, 'rb') as f_in:
    with gzip.open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("✅ تم ضغط الملف وحفظه في:", output_file)

✅ تم ضغط الملف وحفظه في: /kaggle/working/emar_detail.csv.gz


In [4]:
import os

file_path = "/kaggle/working/emar_detail.csv.gz"

file_size_bytes = os.path.getsize(file_path)
file_size_mb = file_size_bytes / (1024 ** 2)

print(f"📦 حجم الملف: {file_size_mb:.2f} ميغابايت ({file_size_bytes:,} بايت)")


📦 حجم الملف: 433.66 ميغابايت (454,724,787 بايت)


قراءة الملف المضغوط

In [3]:
import gzip
import time
import psutil

file_path_gz = "/kaggle/working/emar_detail.csv.gz"

start_time = time.time()
start_memory = psutil.Process().memory_info().rss / (1024 ** 2)

row_count = 0

with gzip.open(file_path_gz, 'rt') as f: 
    for _ in f:
        row_count += 1

end_time = time.time()
end_memory = psutil.Process().memory_info().rss / (1024 ** 2)

execution_time = end_time - start_time
used_memory = end_memory - start_memory

print(f" عدد الصفوف الفعلية : {row_count - 1:,}")
print(f"\n⏱️ وقت التنفيذ الكلي: {execution_time:.2f} ثانية")
print(f"💾 الذاكرة المستخدمة أثناء التنفيذ: {used_memory:.2f} ميغابايت")


 عدد الصفوف الفعلية : 54,514,587

⏱️ وقت التنفيذ الكلي: 33.11 ثانية
💾 الذاكرة المستخدمة أثناء التنفيذ: 0.12 ميغابايت


عرض النتائج

In [6]:
import pandas as pd

data = {
    "Method": ["Chunksize", "Dask", "Compression (.gz)"],
    "Time (seconds)": [213.43, 197.25, 33.11 ],
    "Memory (megabytes)": [29.78, 1845.58, 0.12]
}


results = pd.DataFrame(data)


print("📊 مقارنة أداء طرق قراءة الملفات الكبيرة:\n")
print(results.to_string(index=False))


📊 مقارنة أداء طرق قراءة الملفات الكبيرة:

           Method  Time (seconds)  Memory (megabytes)
        Chunksize          213.43               29.78
             Dask          197.25             1845.58
Compression (.gz)           33.11                0.12
